In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
import pickle


In [3]:
## Load the dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Preprocess the data
## Drop irrelevant columns (RowNumber,CustomerId,Surname)
data = data.drop(["RowNumber","CustomerId","Surname"], axis=1)
data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## Encode Categorical Variables
label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])
data.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
6,822,France,1,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,0,29,4,115046.74,4,1,0,119346.88,1
8,501,France,1,44,4,142051.07,2,0,1,74940.50,0
9,684,France,1,27,2,134603.88,1,1,1,71725.73,0


In [6]:
## OneHot Encode "Geography" column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[["Geography"]])
geo_encoder

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [8]:
onehot_encoder_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [14]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))

geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [15]:
## Combine one hot encoded columns with original data

data = pd.concat([data.drop("Geography",axis=1),geo_encoded_df] ,axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [16]:
## Save the encoders and scalers 

with open("lable_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender , file)

with open("onehot_encoder_geo.pkl","wb") as file:
    pickle.dump(onehot_encoder_geo , file)

In [20]:
## Divide the dataset into dependent and independent features
X = data.drop("Exited",axis=1)
y = data["Exited"]


## Split the data into training and testing sets
X_train , X_test , y_train , y_test = train_test_split(X,y , test_size=.2 , random_state=42)


## Scale this features 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [22]:
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler ,file)

### ANN Implementation

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime



In [25]:
## Build out ANN Model

model = Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)), ## HL1 connected with input layer
    Dense(32,activation="relu"), ##HL2
    Dense(1,activation="sigmoid")  ##Output layer
])


In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)

In [31]:
## Compile the model
model.compile(optimizer=opt , loss="binary_crossentropy",metrics=["accuracy"])

In [46]:
## Set up the Tensorbaord

from tensorflow.keras.callbacks import EarlyStopping , TensorBoard


log_dir="logs/fit/" +datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir , histogram_freq=1)


In [47]:
## Set up Early stopping 

early_stopping_callback = EarlyStopping(monitor="val_loss", patience=10 ,restore_best_weights=True)

In [48]:
## Train the model

history = model.fit(
    X_train ,y_train, validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback , early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3565 - accuracy: 0.8574 - val_loss: 0.3472 - val_accuracy: 0.8635
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3440 - accuracy: 0.8599 - val_loss: 0.3491 - val_accuracy: 0.8600
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3407 - accuracy: 0.8600 - val_loss: 0.3400 - val_accuracy: 0.8610
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3359 - accuracy: 0.8601 - val_loss: 0.3415 - val_accuracy: 0.8605
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3354 - accuracy: 0.8608 - val_loss: 0.3533 - val_accuracy: 0.8540
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3313 - accuracy: 0.8627 - val_loss: 0.3475 - val_accuracy: 0.8535
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3300 - accuracy: 0.8622 - val_loss: 0.3428 - val_accuracy: 0.8580

In [49]:
model.save("model.h5")

d:\Code\ANN Project Implementation\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [50]:
## Load Tensorboard Extension

%reload_ext tensorboard

In [51]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 23680), started 0:13:48 ago. (Use '!kill 23680' to kill it.)